In [1]:
# Inicie uma sessão Spark no seu notebook.
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ProductDataAnalysis").getOrCreate()

In [4]:
# Carregue os dados do arquivo CSV que já está dentro do seu projeto
file_path = "product+classification+and+clustering/pricerunner_aggregate.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [5]:
df.show(5)

+----------+--------------------+------------+-----------+--------------------+------------+---------------+
|Product ID|       Product Title| Merchant ID| Cluster ID|       Cluster Label| Category ID| Category Label|
+----------+--------------------+------------+-----------+--------------------+------------+---------------+
|         1|apple iphone 8 pl...|           1|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         2|apple iphone 8 pl...|           2|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         3|apple mq8n2b/a ip...|           3|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         4|apple iphone 8 pl...|           4|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         5|apple iphone 8 pl...|           5|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
+----------+--------------------+------------+-----------+--------------------+------------+---------------+
only showing top 5 

In [6]:
df.printSchema()

root
 |-- Product ID: integer (nullable = true)
 |-- Product Title: string (nullable = true)
 |--  Merchant ID: integer (nullable = true)
 |--  Cluster ID: integer (nullable = true)
 |--  Cluster Label: string (nullable = true)
 |--  Category ID: integer (nullable = true)
 |--  Category Label: string (nullable = true)



In [7]:
df.describe().show()

+-------+-----------------+--------------------+------------------+------------------+--------------------+------------------+----------------+
|summary|       Product ID|       Product Title|       Merchant ID|        Cluster ID|       Cluster Label|       Category ID|  Category Label|
+-------+-----------------+--------------------+------------------+------------------+--------------------+------------------+----------------+
|  count|            35311|               35311|             35311|             35311|               35311|             35311|           35311|
|   mean|26150.80017558268|                null|120.50188326583785|30110.687632749003|                null| 2618.142929965167|            null|
| stddev|13498.19122018199|                null|117.04555721851422|18410.265642128295|                null|3.6007080727956833|            null|
|    min|                1|10 hd portable te...|                 1|                 1|AEG A71101TSX0 St...|              2612|          